In [1]:
from gurobipy import *
import os
import csv

In [2]:
#os.getcwd()

In [5]:
os.chdir("C:\\Users\\yash1\\OneDrive - Oklahoma A and M System\\MAIN DRIVE\\Semester 3\\IEM 5013 (Introduction to optimization)\\Project_Final")

In [6]:
name1 = "ZJX_Central.csv"
name2 = "ZJX_East.csv"
name3 = "ZJX_North.csv"
name4 = "ZJX_South.csv"
name5 = "ZJX_West.csv"

with open(name1, 'r') as file:
    csv_reader = csv.reader(file)
    data = []

    for row in csv_reader:
        data.extend(row)

data_file_name = file.name
#print(data)

In [7]:
s = [32, 40]
r = [int(value) for value in data]

## limit input size for trial runs
#r = r[0:240]

In [8]:
s

[32, 40]

In [9]:
# Model Initiation
m = Model("Shift_Scheduling_Problem")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-06


In [11]:
######################################
########## Input parameters ##########
######################################
## Subscript 1 is for 8 hours shifts and 2 is for 10 hour shifts

# HOURLY cost of 1 shift
hcost = 1

# hours in 1 shift
h1 = 8
h2 = 10

# number of shifts in schedule
j1 = 10
j2 = 8

# cost of entire schedule
c1 = hcost*h1*j1
c2 = hcost*h2*j2

# overtime hourly cost
ot_hcost = 1.5 * hcost

# total number of schedules
n1 = 25
n2 = 25

In [12]:
i_indices = range(n1) #Total count of standard schedules
k_indices = range(n2) #Total count of compressed schedules

j1_indices = range(j1) #shift indices in standard schedule
j2_indices = range(j2) #shift indices in compressed schedule

t_indices = range(len(r)) #time period indices

In [13]:
########################################
########## Decision Variables ##########
########################################

###
### Schedule selection ###
S = m.addVars(i_indices, vtype=GRB.BINARY, name="S")
C = m.addVars(k_indices, vtype=GRB.BINARY, name="C")


###
### Normal Working Hours ### (only captures starting marks)
# Normal working hours initiation for each shift "j1" in standard schedule "i"
X = m.addVars(i_indices, t_indices, vtype=GRB.BINARY, name="X")

# Normal working hours initiation for each shift "j2" in compressed schedule "k"
Y = m.addVars(k_indices, t_indices, vtype=GRB.BINARY, name="Y")


###
### Overtime working hours ### (captures all the time periods that are active)
# Active period for Overtime in "i"th standard schedule's "j1"th shift
OS = m.addVars(i_indices, t_indices, vtype=GRB.BINARY, name="OS")

# Active period for Overtime in "k"th compressed schedule's "j2"th shift
OC = m.addVars(k_indices, t_indices, vtype=GRB.BINARY, name="OC")

In [14]:
########################################
########## Objective function ##########
########################################

expression01 = quicksum(c1 * S[i] for i in i_indices)
expression02 = quicksum(c2 * C[k] for k in k_indices)
expression03 = quicksum(ot_hcost * OS[i, t] for i in i_indices for t in t_indices)
expression04 = quicksum(ot_hcost * OC[k, t] for k in k_indices for t in t_indices)

# Set objective function
m.setObjective(expression01 + expression02 + expression03 + expression04, GRB.MINIMIZE)

In [15]:
#################################
########## Constraints ##########
#################################

###
### demand constraint

## POSSIBLE ERROR

for t in t_indices:
    time_range_normal = range(max(0, t-31), t+1)
    time_range_compressed = range(max(0, t-39), t+1)
    
    exp1 = quicksum(X[i, tau] for i in i_indices for tau in time_range_normal)
    exp2 = quicksum(Y[k, tau] for k in k_indices for tau in time_range_compressed)
    
    exp3 = quicksum(OS[i, t] for i in i_indices)
    exp4 = quicksum(OC[k, t] for k in k_indices)
    
    m.addConstr(exp1 + exp2 + exp3 + exp4 >= r[t], name=f"demand_constraint_{t}")

In [16]:
###
### OT continuation constraints
### Only gets affected by same values of "j"
for i in i_indices:
    for t in t_indices:
        if t >= s[0]:
            m.addConstr(OS[i, t] <= X[i, t-s[0]] + OS[i, t-1], name=f"OS_Cont_{i},{t}")

for k in k_indices:
    for t in t_indices:
        if t >= s[1]:
            m.addConstr(OC[k, t] <= Y[k, t-s[1]] + OC[k, t-1], name=f"OC_Cont_{k},{t}")

In [17]:
###
### 8-hours gap
### affected by other "j" of same "i"
# X & OS
for i in i_indices:
    for t in t_indices:
        time_check_range1 = range(max(0,t-63), t)           
        exp001 = quicksum(X[i, tau1] for tau1 in time_check_range1)
            
        time_check_range2 = range(max(0,t-32), t)           
        exp002 = quicksum(OS[i, tau2] for tau2 in time_check_range2)
            
        m.addConstr(100000*(1 - X[i, t]) >= exp001 + exp002, name=f"8hourGap_OS_{i}{t}")
            
# Y & OC
for k in k_indices:
    for t in t_indices:
        time_check_range1 = range(max(0,t-71), t)           
        exp101 = quicksum(Y[k, tau1] for tau1 in time_check_range1)
            
        time_check_range2 = range(max(0,t-32), t)           
        exp102 = quicksum(OC[k, tau2] for tau2 in time_check_range2)
            
        m.addConstr(100000*(1 - Y[k, t]) >= exp101 + exp102, name=f"8hourGap_OC_{k}{t}")
            

In [18]:
###
### Shift validation constraint
for i in i_indices:
    a = quicksum(X[i, t] for t in t_indices)
    b = 10*S[i]
    m.addConstr(a == b, name=f"shift_validation_S{i}")

for k in k_indices:
    a = quicksum(Y[k, t] for t in t_indices)
    b = 8*C[k]
    m.addConstr(a == b, name=f"shift_validation_C{k}")

In [19]:
###
### Last few shifts should not be initialized
for t in range(len(t_indices)-31, len(t_indices)):
    a = quicksum(X[i, t] for i in i_indices)
    m.addConstr(a == 0, name=f"last_zeroes_8hours_{t}")

for t in range(len(t_indices)-39, len(t_indices)):
    a = quicksum(Y[k, t] for k in k_indices)
    m.addConstr(a == 0, name=f"last_zeroes_10hours_{t}")

In [20]:
###
### OT can't start in initial time
for t in range(0, 32):
    a = quicksum(OS[i, t] for i in i_indices)
    m.addConstr(a == 0, name=f"Initial_OS_zero_{t}")
    
for t in range(0, 40):
    a = quicksum(OC[k, t] for k in k_indices)
    m.addConstr(a == 0, name=f"Initial_OC_zero_{t}")

In [21]:
###
###
#for i in i_indices:
#    for t in t_indices:
#        a = quicksum(X[i, t])
#        m.addConstr(a <= 1, name=f"one_shift_at_a_time_S_{i}{t}")
#
#for k in k_indices:
#    for t in t_indices:
#        a = quicksum(Y[k, t])
#        m.addConstr(a <= 1, name=f"one_shift_at_a_time_C_{k}{t}")   

In [22]:
m.Params.timeLimit = 0.25*3600

m.optimize()

Set parameter TimeLimit to value 900
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 134136 rows, 134450 columns and 9300800 nonzeros
Model fingerprint: 0x4fe0a26b
Variable types: 0 continuous, 134450 integer (134450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 3659.0000000
Presolve removed 1962 rows and 3550 columns (presolve time = 5s) ...
Presolve removed 2064 rows and 3550 columns (presolve time = 10s) ...
Presolve removed 2264 rows and 3550 columns (presolve time = 15s) ...
Presolve removed 2269 rows and 3550 columns (presolve time = 20s) ...
Presolve removed 3534 rows and 3550 columns (presolve time = 25s) ..

In [36]:
for i in i_indices:
    for t in t_indices:
        if OS[i, t].x != 0:
            print(f"[{i},{t}]")

[1,1118]
[2,362]
[2,1034]
[2,1150]
[4,433]
[5,1118]
[6,478]
[7,446]
[8,446]
[10,750]


In [40]:
model_status = ""

if m.status == GRB.OPTIMAL:
    model_status += f"Termination status: Optimal\n"
    model_status += f"Best objective value: {m.objVal}\n"
    model_status += f"Elapsed wall-clock time: {m.Runtime} seconds\n"
    model_status += f"Optimality gap at termination: {m.MIPGap}%\n"
elif m.status == GRB.INFEASIBLE:
    model_status += "Termination status: Infeasible\n"
    model_status += f"Infeasibility reason: {m.UnbdRay}\n"
elif m.status == GRB.UNBOUNDED:
    model_status += "Termination status: Unbounded\n"
elif m.status == GRB.TIME_LIMIT:
    model_status += "Termination status: Time Limit Exceeded\n"
    model_status += f"Best objective value: {m.objVal}\n"
    model_status += f"Optimality gap at termination: {m.MIPGap}%\n"
else:
    model_status += f"Optimization terminated with status: {m.status}\n"

print(model_status)

Termination status: Optimal
Best objective value: 2197.5
Elapsed wall-clock time: 242.3090000152588 seconds
Optimality gap at termination: 0.0%



In [41]:
distribution_text = ""

In [42]:
if m.status == GRB.OPTIMAL:
    distribution_text += "*****STANDARD SCHEDULES*****\n\n"
    for i in i_indices:
        distribution_text += f"Schedule {i}:"
        a = 1
        for t in t_indices:
            if X[i, t].x != 0:
                distribution_text += f"\nShift {a} starts at period: {t}"
                a += 1
            elif OS[i, t].x != 0:
                if OS[i, max(0, t-1)].x == 0:
                    distribution_text += f" || overtime starts at {t}"
                else:
                    pass
                
                if OS[i, min(t+1, len(t_indices))].x == 0:
                    distribution_text += f" and ends at {t}"
                else:
                    pass
        distribution_text += "\n"

print(distribution_text)

*****STANDARD SCHEDULES*****

Schedule 0:
Shift 1 starts at period: 126
Shift 2 starts at period: 313
Shift 3 starts at period: 507
Shift 4 starts at period: 736
Shift 5 starts at period: 814
Shift 6 starts at period: 885
Shift 7 starts at period: 988
Shift 8 starts at period: 1115
Shift 9 starts at period: 1219
Shift 10 starts at period: 1309
Schedule 1:
Shift 1 starts at period: 70
Shift 2 starts at period: 157
Shift 3 starts at period: 314
Shift 4 starts at period: 413
Shift 5 starts at period: 540
Shift 6 starts at period: 604
Shift 7 starts at period: 733
Shift 8 starts at period: 989
Shift 9 starts at period: 1086 || overtime starts at 1118 and ends at 1118
Shift 10 starts at period: 1190
Schedule 2:
Shift 1 starts at period: 157
Shift 2 starts at period: 228
Shift 3 starts at period: 330 || overtime starts at 362 and ends at 362
Shift 4 starts at period: 445
Shift 5 starts at period: 524
Shift 6 starts at period: 637
Shift 7 starts at period: 742
Shift 8 starts at period: 1002 |

In [43]:
if m.status == GRB.OPTIMAL:
    distribution_text += "\n\n\n*****COMPRESSED SCHEDULES*****\n\n"
    for k in k_indices:
        distribution_text += f"Schedule {k}:"
        a = 1
        for t in t_indices:
            if Y[k, t].x != 0:
                distribution_text += f"\nShift {a} starts at period: {t}"
                a += 1
            elif OC[k, t].x != 0:
                if OC[k, max(0, t-1)].x == 0:
                    distribution_text += f" || overtime starts at {t}"
                else:
                    pass
                
                if OC[k, min(t+1, len(t_indices))].x == 0:
                    distribution_text += f" and ends at {t}"
                else:
                    pass
        distribution_text += "\n"

print(distribution_text)

*****STANDARD SCHEDULES*****

Schedule 0:
Shift 1 starts at period: 126
Shift 2 starts at period: 313
Shift 3 starts at period: 507
Shift 4 starts at period: 736
Shift 5 starts at period: 814
Shift 6 starts at period: 885
Shift 7 starts at period: 988
Shift 8 starts at period: 1115
Shift 9 starts at period: 1219
Shift 10 starts at period: 1309
Schedule 1:
Shift 1 starts at period: 70
Shift 2 starts at period: 157
Shift 3 starts at period: 314
Shift 4 starts at period: 413
Shift 5 starts at period: 540
Shift 6 starts at period: 604
Shift 7 starts at period: 733
Shift 8 starts at period: 989
Shift 9 starts at period: 1086 || overtime starts at 1118 and ends at 1118
Shift 10 starts at period: 1190
Schedule 2:
Shift 1 starts at period: 157
Shift 2 starts at period: 228
Shift 3 starts at period: 330 || overtime starts at 362 and ends at 362
Shift 4 starts at period: 445
Shift 5 starts at period: 524
Shift 6 starts at period: 637
Shift 7 starts at period: 742
Shift 8 starts at period: 1002 |

In [44]:
file_path = "PatelPatelBhandari_part02.txt"

In [45]:
with open(file_path, "a") as file:
    file.write(f"For data obtained from {data_file_name}:\n\n")
    file.write("Model Status:\n")
    file.write(model_status)
    file.write("\n")
    file.write("Shift_Scheduling_Problem_results:\n")
    file.write(distribution_text)
    file.write("\n\n--XX--XX--XX--XX--XX--XX--XX--XX--XX--XX--XX--XX--XX--\n\n")